In [67]:
import pandas as pd
import geopandas as gpd

from shared_utils import catalog_utils, rt_dates, rt_utils
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
gcsgp = GCSGeoPandas()

# Culver City Runtime Export

Simple example of filtering for single-day speed data over the last 12 months and calculating runtimes. Request from Culver City.

In [2]:
catalog_utils

<module 'shared_utils.catalog_utils' from '/home/jovyan/data-analyses/_shared_utils/shared_utils/catalog_utils.py'>

In [3]:
catalog_utils.get_catalog?

Signature:
catalog_utils.get_catalog(
    catalog_name: Literal['shared_data_catalog', 'gtfs_analytics_data'],
) -> pathlib.Path
Docstring:
Grab either the shared_data_catalog (uses intake driver) or
gtfs_analytics_data catalog (uses OmegaConf yaml parser).
File:      ~/data-analyses/_shared_utils/shared_utils/catalog_utils.py
Type:      function

In [4]:
catalog = catalog_utils.get_catalog('gtfs_analytics_data')

In [5]:
speedmap_dir = catalog.speedmap_segments.dir

In [49]:
analysis_date = rt_dates.DATES['sep2025']

In [50]:
path = f"{speedmap_dir}{catalog.speedmap_segments.segment_timeofday}_{analysis_date}.parquet"
path

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/rollup_singleday/speeds_shape_timeofday_speedmap_segments_2025-09-24.parquet'

In [51]:
speeds = gcsgp.read_parquet(path)

In [52]:
speeds.columns

Index(['schedule_gtfs_dataset_key', 'shape_id', 'shape_array_key', 'route_id',
       'direction_id', 'stop_pair', 'segment_id', 'stop_pair_name',
       'time_of_day', 'p50_mph', 'n_trips', 'p20_mph', 'p80_mph',
       'n_trips_sch', 'trips_hr_sch', 'route_short_name', 'geometry', 'name',
       'base64_url', 'caltrans_district', 'analysis_name', 'source_record_id'],
      dtype='object')

In [53]:
metro_stop_pairs = ['16530__2755', '2755__5222']

In [40]:
all_dates = rt_dates.y2024_dates[8:] + rt_dates.y2025_dates

In [41]:
all_dates

['2024-09-18',
 '2024-10-16',
 '2024-11-13',
 '2024-12-11',
 '2025-01-15',
 '2025-02-12',
 '2025-03-12',
 '2025-04-16',
 '2025-05-14',
 '2025-06-11',
 '2025-07-16',
 '2025-08-20',
 '2025-09-24']

In [55]:
dfs = []
for analysis_date in all_dates:
    path = f"{speedmap_dir}{catalog.speedmap_segments.segment_timeofday}_{analysis_date}.parquet"
    speeds = gcsgp.read_parquet(path, filters=[("name", "==", "LA Metro Bus Schedule")])
    speeds = speeds.query('stop_pair.isin(@metro_stop_pairs)')
    speeds = speeds.assign(analysis_date = analysis_date)
    print(analysis_date)
    print(speeds.shape)
    dfs += [speeds]

2024-09-18
(10, 23)
2024-10-16
(11, 23)
2024-11-13
(11, 23)
2024-12-11
(11, 23)
2025-01-15
(12, 23)
2025-02-12
(12, 23)
2025-03-12
(11, 23)
2025-04-16
(12, 23)
2025-05-14
(11, 23)
2025-06-11
(12, 23)
2025-07-16
(11, 23)
2025-08-20
(11, 23)
2025-09-24
(12, 23)


In [61]:
drop_cols = ['analysis_name', 'source_record_id', 'organization_name',
            'organization_source_record_id']

In [95]:
gdf = pd.concat(dfs).drop(columns=drop_cols)

In [96]:
from calitp_data_analysis.geography_utils import CA_NAD83Albers_m

In [97]:
rt_utils.MPH_PER_MPS

2.237

In [98]:
rt_utils.METERS_PER_MILE

1609.34

In [99]:
gdf = gdf.to_crs(CA_NAD83Albers_m)
gdf['segment_miles'] = gdf.geometry.map(lambda x: x.length / rt_utils.METERS_PER_MILE)

In [100]:
def runtime_from_row(row, col):
    hours = row.segment_miles / row[col]
    seconds = hours * 60**2
    return seconds

In [102]:
gdf = gdf.assign(seconds_p50_speed = gdf.apply(runtime_from_row, axis=1, col='p50_mph'),
                seconds_p20_speed = gdf.apply(runtime_from_row, axis=1, col='p20_mph'),
                seconds_p80_speed = gdf.apply(runtime_from_row, axis=1, col='p80_mph'))

In [103]:
gdf

,schedule_gtfs_dataset_key,shape_id,shape_array_key,route_id,direction_id,stop_pair,segment_id,stop_pair_name,time_of_day,p50_mph,...,route_short_name,name,caltrans_district,base64_url,geometry,analysis_date,segment_miles,seconds_p50_speed,seconds_p20_speed,seconds_p80_speed
3750,0666caf3ec1ecc96b74f4477ee4bc939,350276_JUNE24,707ecc7c9a60834da53fc4c536184ade,35-13183,0.0,16530__2755,16530-2755-1,Washington / Fairfax__Fairfax / Adams,Evening,12.84,...,35/38,LA Metro Bus Schedule,07 - Los Angeles,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (150586.014 -441021.241, 150586.014...",2024-09-18,0.430914,120.816935,233.276608,81.560959
3751,0666caf3ec1ecc96b74f4477ee4bc939,350276_JUNE24,707ecc7c9a60834da53fc4c536184ade,35-13183,0.0,2755__5222,2755-5222-1,Fairfax / Adams__Jefferson / La Cienega,Evening,8.34,...,35/38,LA Metro Bus Schedule,07 - Los Angeles,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (150433.352 -441316.744, 150433.352...",2024-09-18,0.436072,188.232698,192.384890,184.472468
53137,0666caf3ec1ecc96b74f4477ee4bc939,350275_JUNE24,5a6bc72c5f95666b6f8cbd6121276b6c,35-13183,0.0,16530__2755,16530-2755-1,Washington / Fairfax__Fairfax / Adams,Early AM,19.88,...,35/38,LA Metro Bus Schedule,07 - Los Angeles,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (150586.014 -441021.241, 150586.014...",2024-09-18,0.430914,78.032668,78.032668,78.032668
53138,0666caf3ec1ecc96b74f4477ee4bc939,350275_JUNE24,5a6bc72c5f95666b6f8cbd6121276b6c,35-13183,0.0,16530__2755,16530-2755-1,Washington / Fairfax__Fairfax / Adams,Midday,1.01,...,35/38,LA Metro Bus Schedule,07 - Los Angeles,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (150586.014 -441021.241, 150586.014...",2024-09-18,0.430914,1535.930142,2462.364196,135.129742
53139,0666caf3ec1ecc96b74f4477ee4bc939,350275_JUNE24,5a6bc72c5f95666b6f8cbd6121276b6c,35-13183,0.0,16530__2755,16530-2755-1,Washington / Fairfax__Fairfax / Adams,PM Peak,3.58,...,35/38,LA Metro Bus Schedule,07 - Los Angeles,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (150586.014 -441021.241, 150586.014...",2024-09-18,0.430914,433.321074,628.052406,330.061584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622389,d4a07b520153bd5c27860835bc42610e,350287_JUNE25,e50881577fd0bd6c35a271dd4b5e4e1c,35-13188,0.0,2755__5222,2755-5222-1,Fairfax / Adams__Jefferson / La Cienega,Evening,5.99,...,35/38,LA Metro Bus Schedule,07 - Los Angeles / Ventura,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (150433.352 -441316.744, 150433.352...",2025-09-24,0.436072,262.080251,262.080251,262.080251
622390,d4a07b520153bd5c27860835bc42610e,350287_JUNE25,e50881577fd0bd6c35a271dd4b5e4e1c,35-13188,0.0,2755__5222,2755-5222-1,Fairfax / Adams__Jefferson / La Cienega,Midday,6.88,...,35/38,LA Metro Bus Schedule,07 - Los Angeles / Ventura,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (150433.352 -441316.744, 150433.352...",2025-09-24,0.436072,228.177428,323.682620,183.609439
622391,d4a07b520153bd5c27860835bc42610e,350287_JUNE25,e50881577fd0bd6c35a271dd4b5e4e1c,35-13188,0.0,2755__5222,2755-5222-1,Fairfax / Adams__Jefferson / La Cienega,PM Peak,3.18,...,35/38,LA Metro Bus Schedule,07 - Los Angeles / Ventura,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (150433.352 -441316.744, 150433.352...",2025-09-24,0.436072,493.666888,740.500333,327.054314
665067,d4a07b520153bd5c27860835bc42610e,350288_JUNE25,e0e9a22bd1326809333cd877de4ee836,35-13188,0.0,16530__2755,16530-2755-1,Washington / Fairfax__Fairfax / Adams,Evening,9.34,...,35/38,LA Metro Bus Schedule,07 - Los Angeles / Ventura,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (150586.014 -441021.241, 150586.014...",2025-09-24,0.430914,166.090947,166.090947,166.090947


In [107]:
gdf.to_file('metro_38_2024_2025.geojson')

In [108]:
gdf.drop(columns=['geometry']).to_csv('metro_38_2024_2025.csv')